In [107]:
from sklearn.linear_model import Ridge
import joblib
import pandas as pd
import numpy as np

In [108]:
data = pd.read_csv('../datasets/train.csv')
data.head()

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [109]:
X_train_roberta = joblib.load('roberta_mlm_cv_train/roberta_large_tr_p.pkl')
X_train_alberta = joblib.load('albert_mlm_cv_train/albert_xxlarge_v2_tr_p.pkl')
X_train_electra = joblib.load('electra_cv_train/electra_large_tr_p.pkl')
X_train_stacking = np.concatenate((X_train_roberta, X_train_alberta, X_train_electra), axis=1)
X_train_stacking.shape

(2834, 3)

In [110]:
ridge_reg = Ridge(alpha=0.001, max_iter=1000)  # 设置alpha=0.001(Regularization strength)
ridge_reg.fit(X_train_stacking, data['target'].values)

Ridge(alpha=0.001, max_iter=1000)

In [111]:
X_test_roberta = joblib.load('roberta_mlm_cv_train/roberta_large_te_pr.pkl').reshape(7, -1).mean(axis=1)
X_test_alberta = joblib.load('albert_mlm_cv_train/albert_xxlarge_v2_te_pr.pkl').reshape(7, -1).mean(axis=1)
X_test_electra = joblib.load('electra_cv_train/electra_large_te_pr.pkl').reshape(7, -1).mean(axis=1)
X_test_stacking = np.stack((X_test_roberta, X_test_alberta, X_test_electra), axis=1)
X_test_stacking

array([[-1.2130673 , -1.2349802 , -1.0639806 ],
       [-0.48608837, -0.51714957, -0.56315154],
       [-1.1838045 , -1.0915611 , -1.0860678 ],
       [-1.3047185 , -1.3212792 , -1.2881937 ],
       [-0.64509046, -0.65288514, -0.6494516 ],
       [-0.87645596, -0.91668445, -0.83477944],
       [-1.1311338 , -1.2475598 , -1.1726393 ]], dtype=float32)

In [112]:
ridge_reg.predict(X_test_stacking)  # stacking模型融合

array([-0.9621451 , -0.9655635 , -0.9671495 , -0.9493204 , -0.9681693 ,
       -0.9625077 , -0.94282806], dtype=float32)

In [113]:
X_test_roberta * 0.4 + X_test_alberta * 0.3 + X_test_electra * 0.3  # 线性加权融合(可通过optuna包搜索各模型权重(为超参数))

array([-1.1749152 , -0.5185257 , -1.1268104 , -1.3047293 , -0.64873725,
       -0.8760215 , -1.1785133 ], dtype=float32)